## Conv2D - pytorch

#### torch.nn.Conv2d
- ```
    torch.nn.Conv2d(
        in_channels,
        out_channels,
        kernel_size,
        stride=1,
        padding=0,
        dilation=1,
        groups=1,
        bias=Truem
        padding_mode='zeros'
    )
    ```
#### 파라미터 계산
- 커널사이즈^2 * 채널 * 파라미터 + 바이어스(파라미터 수)
- `self.conv1 = nn.Conv2d(1, 6, 5)`
- 채널:1, 파라미터:6, 커널사이즈:5
- 25 * 1 * 6 + 6 = 156

#### 출력 사이즈 계산
- (입력사이즈 + 2*패딩사이즈) - 커널사이즈 / 스트라이드 + 1
- `Input size: (7,7), Kernel size: (3,3) padding: (0,0), stride: (1,1)`
- ( 7 + 2 * 0 ) - 3 / 1 + 1 = 5 -> 출력사이즈: (5,5)
- 패딩이 1이면 입력사이즈와 출력사이즈는 같다.
- `padding: (1,1)` -> 출력사이즈: (7,7)
- **기억해야 될 점은 패딩이 1이 아닌 이상 출력사이즈는 계속 줄어든다.**

#### 맥스풀링
- `torch.nn.functional.max_pool2d(2,2)`
- (커널사이즈, 스트라이드)


In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 입력 이미지 채널 1개, 출력 채널 6개, 5x5의 정사각 컨볼루션 행렬
        # 컨볼루션 커널 정의
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # 아핀(affine) 연산: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5은 이미지 차원에 해당
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # (2, 2) 크기 윈도우에 대해 맥스 풀링(max pooling)
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # 크기가 제곱수라면, 하나의 숫자만을 특정(specify)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # 배치 차원을 제외한 모든 차원을 하나로 평탄화(flatten)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


- summary()를 이용하기 위해서는 torchsummary 모듈이 설치되어 있어야 한다.
- `RuntimeError: input type, weight type`
- 모델을 설정했으면 디바이스와 연결해주어야 한다.


In [8]:
import torchsummary
net.to('cuda')
torchsummary.summary(net,(1,64,64))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (2x2704 and 400x120)

`RuntimeError: mat1 and mat2 shapes cannot be multiplied (2x2704 and 400x120)`
- self.fc1 = nn.Linear(16 * 5 * 5, 120) 이 부분 때문에 오류가 난다.
- (64,64)는 forward를 진행하면서 사이즈가 (5,5)가 되지 않기 때문이다.
- 64 forward process
  - conv1: 60
  - max: 30
  - conv2: 26
  - max: 13
  - fc1: 16 * 13 * 13
- conv1에서 출력사이즈가 60인 이유는 패딩이 디폴트로 0으로 설정되어서, 입력사이즈가 계속 줄어들기 때문이다.

In [9]:
import torchsummary
net.to('cuda')
torchsummary.summary(net,(1,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             156
            Conv2d-2           [-1, 16, 10, 10]           2,416
            Linear-3                  [-1, 120]          48,120
            Linear-4                   [-1, 84]          10,164
            Linear-5                   [-1, 10]             850
Total params: 61,706
Trainable params: 61,706
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.05
Params size (MB): 0.24
Estimated Total Size (MB): 0.29
----------------------------------------------------------------
